In [206]:
import tensorflow as tf
import pandas as pd 
import tensorflow_io as tfio 
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import os
from sklearn.cluster import KMeans

In [207]:
main_dir=("/home/tkrsh/osic-main/")

files=[]

for dirname, _, filenames in os.walk(main_dir):
    for filename in (filenames):
        files.append(os.path.join((dirname), filename))

files=[x for x in files if '.csv' not in x]

train_images= [str(x) for x in files if 'train'  in x]
test_images = [str(x) for x in files if 'test'   in x]

In [208]:
def decode_image(image_path):
    image_bytes = tf.io.read_file(image_path)
    image = tfio.image.decode_dicom_image(image_bytes, dtype=tf.uint16)
    image=np.squeeze(image.numpy())
    return image

def show_scan(image):
    img = decode_image(image)
    patient_name=str(image).split('/')[1]
    fig, ax = plt.subplots()
    im=ax.imshow(img,cmap='Greys')
    plt.axis('off')
    plt.title("Baseline CT Scan of Patient {}".format(patient_name))
    fig.set_size_inches(9,9)
    plt.show()

In [209]:
train=pd.read_csv(main_dir+"train.csv")
df=train.copy()

In [210]:
df.head()

,Patient,Weeks,FVC,Percent,Age,Sex,SmokingStatus
0,ID00007637202177411956430,-4,2315,58.253649,79,Male,Ex-smoker
1,ID00007637202177411956430,5,2214,55.712129,79,Male,Ex-smoker
2,ID00007637202177411956430,7,2061,51.862104,79,Male,Ex-smoker
3,ID00007637202177411956430,9,2144,53.950679,79,Male,Ex-smoker
4,ID00007637202177411956430,11,2069,52.063412,79,Male,Ex-smoker


In [233]:
df=train.copy()

In [234]:
df=pd.concat([df,pd.get_dummies(df['Sex'])],axis=1).drop(['Sex'],axis=1)


In [235]:
df=pd.concat([df,pd.get_dummies(df['SmokingStatus'])],axis=1).drop(['SmokingStatus'],axis=1)


In [236]:
df.head()

,Patient,Weeks,FVC,Percent,Age,Female,Male,Currently smokes,Ex-smoker,Never smoked
0,ID00007637202177411956430,-4,2315,58.253649,79,0,1,0,1,0
1,ID00007637202177411956430,5,2214,55.712129,79,0,1,0,1,0
2,ID00007637202177411956430,7,2061,51.862104,79,0,1,0,1,0
3,ID00007637202177411956430,9,2144,53.950679,79,0,1,0,1,0
4,ID00007637202177411956430,11,2069,52.063412,79,0,1,0,1,0


In [341]:
test.head()

,Weeks,FVC,Percent,Age,Female,Male,Currently smokes,Ex-smoker,Never smoked,dFVC,d%,Gap,Age_Cat_0,Age_Cat_1,Age_Cat_2
0,6,3020,70.186855,73,0.0,0.000000,0.0,0,1,0,1,1,0,0,0
1,15,2739,82.045291,68,-281.0,11.858436,9.0,1,0,0,1,1,0,0,0
2,6,1930,76.672493,73,-809.0,-5.372798,-9.0,0,1,0,1,1,0,0,0
3,17,3294,79.258903,72,1364.0,2.586410,11.0,0,0,1,1,1,0,0,0
4,0,2925,71.824968,73,-369.0,-7.433935,-17.0,0,1,0,1,0,1,0,0


In [314]:
test['dFVC'] =test["FVC"]-test["FVC"].shift(1)
test['d%'] = test["Percent"]-test["Percent"].shift(1)
test['Gap'] = test["Weeks"]-test["Weeks"].shift(1)

In [353]:
test.head()

,Patient,Weeks,FVC,Percent,Age,Female,Male,Currently smokes,Ex-smoker,Never smoked
0,ID00419637202311204720264,6,3020,70.186855,73,0,1,0,1,0
1,ID00421637202311550012437,15,2739,82.045291,68,0,1,0,1,0
2,ID00422637202311677017371,6,1930,76.672493,73,0,1,0,1,0
3,ID00423637202312137826377,17,3294,79.258903,72,0,1,0,1,0
4,ID00426637202313170790466,0,2925,71.824968,73,0,1,0,0,1


In [316]:
test.fillna(0,inplace=True)

In [359]:
test['dFVC'] =test["FVC"]-test["FVC"].shift(1)
test['d%'] = test["Percent"]-test["Percent"].shift(1)
test['Gap'] = test["Weeks"]-test["Weeks"].shift(1)


In [360]:
test.head()

,Patient,Weeks,FVC,Percent,Age,Female,Male,Currently smokes,Ex-smoker,Never smoked,dFVC,d%,Gap
0,ID00419637202311204720264,6,3020,70.186855,73,0,1,0,1,0,NaN,NaN,NaN
1,ID00421637202311550012437,15,2739,82.045291,68,0,1,0,1,0,-281.0,11.858436,9.0
2,ID00422637202311677017371,6,1930,76.672493,73,0,1,0,1,0,-809.0,-5.372798,-9.0
3,ID00423637202312137826377,17,3294,79.258903,72,0,1,0,1,0,1364.0,2.586410,11.0
4,ID00426637202313170790466,0,2925,71.824968,73,0,1,0,0,1,-369.0,-7.433935,-17.0


In [237]:
df['dFVC'] =df["FVC"]-df["FVC"].shift(1)

In [238]:
df['d%'] = df["Percent"]-df["Percent"].shift(1)

In [239]:
df['Gap'] = df["Weeks"]-df["Weeks"].shift(1)

In [240]:
df.fillna(0,inplace=True)

In [358]:
train_df.head()

,Weeks,FVC,Percent,Age,Female,Male,Currently smokes,Ex-smoker,Never smoked,dFVC,d%,Gap,Age_Cat_0,Age_Cat_1,Age_Cat_2
0,-4,2315,58.253649,79,0,1,0,1,0,0.0,0.000000,0.0,0,1,0
1,5,2214,55.712129,79,0,1,0,1,0,-101.0,-2.541520,9.0,0,1,0
2,7,2061,51.862104,79,0,1,0,1,0,-153.0,-3.850025,2.0,0,1,0
3,9,2144,53.950679,79,0,1,0,1,0,83.0,2.088576,2.0,0,1,0
4,11,2069,52.063412,79,0,1,0,1,0,-75.0,-1.887267,2.0,0,1,0


In [363]:
test=pd.concat([test,pd.get_dummies(test['Age_Cat'],prefix="Age_Cat")],axis=1).drop(['Age_Cat'],axis=1)


In [347]:
test=pd.concat([test,pd.get_dummies(test['Sex'])],axis=1).drop(['Sex'],axis=1)


In [350]:
test=pd.concat([test,pd.get_dummies(test['SmokingStatus'])],axis=1).drop(['SmokingStatus'],axis=1)


In [365]:
test=test.drop("Patient",axis=1)

In [366]:
test.head()

,Weeks,FVC,Percent,Age,Female,Male,Currently smokes,Ex-smoker,Never smoked,dFVC,d%,Gap,Age_Cat_0,Age_Cat_1,Age_Cat_2
0,6,3020,70.186855,73,0,1,0,1,0,NaN,NaN,NaN,0,1,0
1,15,2739,82.045291,68,0,1,0,1,0,-281.0,11.858436,9.0,1,0,0
2,6,1930,76.672493,73,0,1,0,1,0,-809.0,-5.372798,-9.0,0,1,0
3,17,3294,79.258903,72,0,1,0,1,0,1364.0,2.586410,11.0,0,0,1
4,0,2925,71.824968,73,0,1,0,0,1,-369.0,-7.433935,-17.0,0,1,0


In [368]:
train_df.head()

,Weeks,FVC,Percent,Age,Female,Male,Currently smokes,Ex-smoker,Never smoked,dFVC,d%,Gap,Age_Cat_0,Age_Cat_1,Age_Cat_2
0,-4,2315,58.253649,79,0,1,0,1,0,0.0,0.000000,0.0,0,1,0
1,5,2214,55.712129,79,0,1,0,1,0,-101.0,-2.541520,9.0,0,1,0
2,7,2061,51.862104,79,0,1,0,1,0,-153.0,-3.850025,2.0,0,1,0
3,9,2144,53.950679,79,0,1,0,1,0,83.0,2.088576,2.0,0,1,0
4,11,2069,52.063412,79,0,1,0,1,0,-75.0,-1.887267,2.0,0,1,0


In [380]:
x_test=test.drop("FVC",axis=1)

In [381]:
y_test=test["FVC"]

In [371]:
x_train.head()

,Weeks,Percent,Age,Female,Male,Currently smokes,Ex-smoker,Never smoked,dFVC,d%,Gap,Age_Cat_0,Age_Cat_1,Age_Cat_2
0,-4,58.253649,79,0,1,0,1,0,0.0,0.000000,0.0,0,1,0
1,5,55.712129,79,0,1,0,1,0,-101.0,-2.541520,9.0,0,1,0
2,7,51.862104,79,0,1,0,1,0,-153.0,-3.850025,2.0,0,1,0
3,9,53.950679,79,0,1,0,1,0,83.0,2.088576,2.0,0,1,0
4,11,52.063412,79,0,1,0,1,0,-75.0,-1.887267,2.0,0,1,0


In [372]:
y_train.head()

0    2315
1    2214
2    2061
3    2144
4    2069
Name: FVC, dtype: int64

In [373]:
x_test.head()

,Weeks,Percent,Age,Female,Male,Currently smokes,Ex-smoker,Never smoked,dFVC,d%,Gap,Age_Cat_0,Age_Cat_1,Age_Cat_2
0,6,70.186855,73,0,1,0,1,0,NaN,NaN,NaN,0,1,0
1,15,82.045291,68,0,1,0,1,0,-281.0,11.858436,9.0,1,0,0
2,6,76.672493,73,0,1,0,1,0,-809.0,-5.372798,-9.0,0,1,0
3,17,79.258903,72,0,1,0,1,0,1364.0,2.586410,11.0,0,0,1
4,0,71.824968,73,0,1,0,0,1,-369.0,-7.433935,-17.0,0,1,0


In [374]:
y_test.head()

0    3020
1    2739
2    1930
3    3294
4    2925
Name: FVC, dtype: int64

In [346]:
test["Female"]=[0,0,0,0,0]

In [349]:
test["Currently smokes"]=[0,0,0,0,0]

In [351]:
train_df.head()

,Weeks,FVC,Percent,Age,Female,Male,Currently smokes,Ex-smoker,Never smoked,dFVC,d%,Gap,Age_Cat_0,Age_Cat_1,Age_Cat_2
0,-4,2315,58.253649,79,0,1,0,1,0,0.0,0.000000,0.0,0,1,0
1,5,2214,55.712129,79,0,1,0,1,0,-101.0,-2.541520,9.0,0,1,0
2,7,2061,51.862104,79,0,1,0,1,0,-153.0,-3.850025,2.0,0,1,0
3,9,2144,53.950679,79,0,1,0,1,0,83.0,2.088576,2.0,0,1,0
4,11,2069,52.063412,79,0,1,0,1,0,-75.0,-1.887267,2.0,0,1,0


In [ ]:
test.columns=[Weeks	,FVC	Percent	Age	Female	Male	Currently smokes	Ex-smoker	Never smoked	dFVC	d%	Gap	Age_Cat_0	Age_Cat_1	Age_Cat_2]

In [338]:
np.array(train_df.columns)

array(['Weeks', 'FVC', 'Percent', 'Age', 'Female', 'Male',
       'Currently smokes', 'Ex-smoker', 'Never smoked', 'dFVC', 'd%',
       'Gap', 'Age_Cat_0', 'Age_Cat_1', 'Age_Cat_2'], dtype=object)

In [332]:
test=test.drop("Patient",axis=1)

In [331]:
train_df.head()

,Weeks,FVC,Percent,Age,Female,Male,Currently smokes,Ex-smoker,Never smoked,dFVC,d%,Gap,Age_Cat_0,Age_Cat_1,Age_Cat_2
0,-4,2315,58.253649,79,0,1,0,1,0,0.0,0.000000,0.0,0,1,0
1,5,2214,55.712129,79,0,1,0,1,0,-101.0,-2.541520,9.0,0,1,0
2,7,2061,51.862104,79,0,1,0,1,0,-153.0,-3.850025,2.0,0,1,0
3,9,2144,53.950679,79,0,1,0,1,0,83.0,2.088576,2.0,0,1,0
4,11,2069,52.063412,79,0,1,0,1,0,-75.0,-1.887267,2.0,0,1,0


In [242]:
df_1=df[df['Patient']=="ID00007637202177411956430"]

In [243]:
Means=KMeans(n_clusters=3).fit((df["Age"].values).reshape(-1,1))

In [244]:
df["Age_Cat"]=Means.labels_

In [361]:
test["Age_Cat"]=Means.fit_predict((test["Age"].values).reshape(-1,1))

In [247]:
df=pd.concat([df,pd.get_dummies(df['Age_Cat'],prefix="Age_Cat")],axis=1).drop(['Age_Cat'],axis=1)


In [470]:
df.head()

,Patient,Weeks,FVC,Percent,Age,Female,Male,Currently smokes,Ex-smoker,Never smoked,dFVC,d%,Gap,Age_Cat_0,Age_Cat_1,Age_Cat_2
0,ID00007637202177411956430,-4,2315,58.253649,79,0,1,0,1,0,0.0,0.000000,0.0,0,1,0
1,ID00007637202177411956430,5,2214,55.712129,79,0,1,0,1,0,-101.0,-2.541520,9.0,0,1,0
2,ID00007637202177411956430,7,2061,51.862104,79,0,1,0,1,0,-153.0,-3.850025,2.0,0,1,0
3,ID00007637202177411956430,9,2144,53.950679,79,0,1,0,1,0,83.0,2.088576,2.0,0,1,0
4,ID00007637202177411956430,11,2069,52.063412,79,0,1,0,1,0,-75.0,-1.887267,2.0,0,1,0


In [471]:
train_df=df.drop("Patient",axis=1)

In [479]:
df["Gap"]= [int(x) for x in df["Gap"]]

In [480]:
df.head()

,Patient,Weeks,FVC,Percent,Age,Female,Male,Currently smokes,Ex-smoker,Never smoked,dFVC,d%,Gap,Age_Cat_0,Age_Cat_1,Age_Cat_2
0,ID00007637202177411956430,-4,2315,58.253649,79,0,1,0,1,0,0.0,0.000000,0,0,1,0
1,ID00007637202177411956430,5,2214,55.712129,79,0,1,0,1,0,-101.0,-2.541520,9,0,1,0
2,ID00007637202177411956430,7,2061,51.862104,79,0,1,0,1,0,-153.0,-3.850025,2,0,1,0
3,ID00007637202177411956430,9,2144,53.950679,79,0,1,0,1,0,83.0,2.088576,2,0,1,0
4,ID00007637202177411956430,11,2069,52.063412,79,0,1,0,1,0,-75.0,-1.887267,2,0,1,0


In [473]:
def score(y_true, y_pred):
    tf.dtypes.cast(y_true, tf.float32)
    tf.dtypes.cast(y_pred, tf.float32)
    sigma = y_pred[:, 2] - y_pred[:, 0]
    fvc_pred = y_pred[:, 1]
    sigma_clip = tf.maximum(sigma, C1)
    delta = tf.abs(y_true[:, 0] - fvc_pred)
    delta = tf.minimum(delta, C2)
    sq2 = tf.sqrt( tf.dtypes.cast(2, dtype=tf.float32) )
    metric = (delta / sigma_clip)*sq2 + tf.math.log(sigma_clip* sq2)
    return K.mean(metric)

In [476]:
from sklearn.model_selection import KFold
import tensorflow as tf

In [482]:
# PARAMETERS
n_folds=5

In [483]:
fold=KFold(n_splits=n_folds)

In [488]:
for train , test in fold.split(df):
  

SyntaxError: unexpected EOF while parsing (<ipython-input-488-2996885ce8b2>, line 2)